In [33]:
import os
import re
import xml.etree.ElementTree as ET
import time
import glob
from PIL import Image
import numpy as np
import torch
import faiss # Using CPU version now
from tqdm import tqdm
import gc
import math
import random
from torch.utils.data import DataLoader as TorchDataLoader
import pandas as pd
import base64
import io
import json # For embedding JS data and parsing LLM output


In [34]:

# Ollama Client Library
try:
    import ollama
    OLLAMA_AVAILABLE = True
except ImportError:
    print("WARNING: Ollama library not found. Triple extraction/generation skipped. Install with: pip install ollama")
    ollama = None
    OLLAMA_AVAILABLE = False

# Hugging Face Libraries
from transformers import (
    AutoProcessor, AutoModel, AutoTokenizer,
    pipeline, BitsAndBytesConfig
)
# Sentence Transformers no longer needed for fine-tuning

# Scikit-learn for cosine similarity
from sklearn.metrics.pairwise import cosine_similarity


# Configuration
# ------------------------------------------
CONFIG = {
    "scan_dir": r"D:\NLP apps\Scans",
    "report_dir": r"D:\NLP apps\Reports",
    "num_reports_to_process": 150,  # <<< START EXTREMELY SMALL (e.g., 50-100) >>>
    "max_reports_total": 3999,
    "output_dir": r"D:\NLP apps\radiology_rag_kg_vis_output_v2", # New output dir

    # --- Evaluation ---
    "num_reports_to_evaluate": 100, # <<< Number of reports to run evaluation on >>>
    "eval_graph_sim_floor": 0.6, # <<< Threshold for graph similarity floor >>>

    # --- Triple Extraction (using Ollama Llama3) ---
    "triple_extractor_model": "llama3", # Ollama model for extraction
    "ollama_base_url": "http://localhost:11434", # Default Ollama API endpoint

    # --- Embedding Model (CLIP for Retrieval & Evaluation) ---
    "embedding_model_name": "openai/clip-vit-base-patch32", # Use CLIP

    # --- RAG Components ---
    # Generator Model (Using Ollama)
    "generator_type": "ollama",
    "ollama_generator_model": "llava-llama3", # Separate model for generation
    "ollama_num_ctx": 4096,
    # Retrieval
    "top_k_retrieval": 3,

    # --- Evaluation Metrics ---
    "eval_embedding_weight": 0.7, # Weight for embedding similarity in combined score
    "eval_graph_similarity_weight": 0.3, # Weight for graph similarity

    # --- Hardware ---
    "use_gpu": torch.cuda.is_available(),
    "embedding_device": "cuda" if torch.cuda.is_available() else "cpu",
    "faiss_use_gpu": False, # Sticking to CPU Faiss
    "generator_device": "cpu", # Ollama runs externally
    "triple_extractor_device": "cpu", # Ollama runs externally
}
# ------------------------------------------


In [35]:

# --- Helper Functions ---
def cleanup_memory():
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()

def extract_text_from_xml(xml_path):
    try:
        tree = ET.parse(xml_path); root = tree.getroot(); texts = []
        for tag in ['AbstractText', 'FINDINGS', 'IMPRESSION', 'REPORT_TEXT', 'paragraph']:
             for elem in root.findall(f'.//{tag}'):
                 if elem.text:
                     cleaned_text = re.sub(r'\s+', ' ', elem.text.strip())
                     if cleaned_text: texts.append(cleaned_text)
        if not texts:
             all_text = ' '.join(node.text.strip() for node in root.iter() if node.text and node.text.strip())
             if all_text: all_text = re.sub(r'\s+', ' ', all_text).strip(); texts.append(all_text)
        full_text = "\n".join(texts); return re.sub(r'\s+', ' ', full_text).strip()
    except Exception: return None

def find_images_for_report(report_id, scan_dir):
    front_pattern = os.path.join(scan_dir, f"CXR{report_id}_*_IM-*-4*.[Pp][Nn][Gg]")
    side_pattern = os.path.join(scan_dir, f"CXR{report_id}_*_IM-*-3*.[Pp][Nn][Gg]")
    front_images = glob.glob(front_pattern); side_images = glob.glob(side_pattern)
    front_image_path = front_images[0] if front_images else None
    side_image_path = side_images[0] if side_images else None
    if not front_image_path and not side_image_path:
         any_pattern = os.path.join(scan_dir, f"CXR{report_id}_*.[Pp][Nn][Gg]")
         any_images = sorted(glob.glob(any_pattern))
         if len(any_images) >= 1: front_image_path = any_images[0]
         if len(any_images) >= 2: side_image_path = any_images[1]
         if not front_image_path and not side_image_path:
            jpg_pattern = os.path.join(scan_dir, f"CXR{report_id}_*.[Jj][Pp][Gg]")
            jpeg_pattern = os.path.join(scan_dir, f"CXR{report_id}_*.[Jj][Pp][Ee][Gg]")
            jpg_images = sorted(glob.glob(jpg_pattern) + glob.glob(jpeg_pattern))
            if len(jpg_images) >= 1: front_image_path = jpg_images[0]
            if len(jpg_images) >= 2: side_image_path = jpg_images[1]
    return front_image_path, side_image_path

def encode_image_to_base64(image_path):
    try:
        with Image.open(image_path) as img:
            if img.mode != 'RGB': img = img.convert('RGB')
            buffer = io.BytesIO(); img.save(buffer, format="JPEG")
            img_bytes = buffer.getvalue(); base64_string = base64.b64encode(img_bytes).decode('utf-8')
            return base64_string
    except Exception as e: print(f"Error encoding image {image_path}: {e}"); return None

def calculate_graph_similarity(triples1, triples2):
    """ Calculates simple Jaccard similarity based on entities and predicates. """
    if not triples1 or not triples2: return 0.0
    entities1 = set(s for s,p,o in triples1) | set(o for s,p,o in triples1)
    entities2 = set(s for s,p,o in triples2) | set(o for s,p,o in triples2)
    predicates1 = set(p for s,p,o in triples1)
    predicates2 = set(p for s,p,o in triples2)
    entity_intersect = len(entities1.intersection(entities2)); entity_union = len(entities1.union(entities2))
    entity_sim = entity_intersect / entity_union if entity_union > 0 else 0
    predicate_intersect = len(predicates1.intersection(predicates2)); predicate_union = len(predicates1.union(predicates2))
    predicate_sim = predicate_intersect / predicate_union if predicate_union > 0 else 0
    return (entity_sim + predicate_sim) / 2.0


In [36]:

# --- Core Classes ---
class DataLoader:
    """Loads reports and associated images."""
    def __init__(self, report_dir, scan_dir, num_to_load, max_total):
        self.report_dir = report_dir; self.scan_dir = scan_dir
        self.num_to_load = min(num_to_load, max_total); self.max_total = max_total

    def load_data(self):
        """Loads report texts, IDs, and image paths."""
        data = []; report_files = sorted(glob.glob(os.path.join(self.report_dir, "*.[Xx][Mm][Ll]")))
        if not report_files: raise FileNotFoundError(f"No XML reports found in {self.report_dir}")
        print(f"Found {len(report_files)} reports. Processing up to {self.num_to_load}...")
        processed_count, skipped_count = 0, 0
        for report_path in tqdm(report_files, desc="Loading Reports"):
            if processed_count >= self.num_to_load: break
            report_filename = os.path.basename(report_path)
            report_id_match = re.match(r"(\d+)\.[Xx][Mm][Ll]", report_filename, re.IGNORECASE)
            if not report_id_match: continue
            report_id = report_id_match.group(1); report_text = extract_text_from_xml(report_path)
            if report_text:
                front_img, side_img = find_images_for_report(report_id, self.scan_dir)
                if front_img: # Require at least front image
                    data.append({"report_id": report_id, "report_path": report_path, "report_text": report_text,
                                 "front_image_path": front_img, "side_image_path": side_img})
                    processed_count += 1
                else: skipped_count += 1
            else: skipped_count += 1
        print(f"Successfully loaded {len(data)} reports with front images. Skipped {skipped_count}.")
        if len(data) < self.num_to_load: print(f"Warning: Loaded fewer reports ({len(data)}) than requested.")
        return data


In [37]:

class TripleExtractor:
    """ Extracts triples using an Ollama model. """
    def __init__(self, config):
        self.config = config
        self.model_name = config["triple_extractor_model"]
        self.base_url = config.get("ollama_base_url", "http://localhost:11434")
        self.client = None
        self._initialize_client()

    def _initialize_client(self):
        """Initializes Ollama client."""
        if not OLLAMA_AVAILABLE: print("Ollama library not available for Triple Extractor."); return
        try:
            print(f"Initializing Ollama client for Triple Extraction (Model: '{self.model_name}') at {self.base_url}...")
            self.client = ollama.Client(host=self.base_url); self.client.list()
            print("Ollama client for Triple Extraction initialized.")
            available_models = [m['name'] for m in self.client.list()['models']]
            if not any(m.startswith(self.model_name) for m in available_models):
                 print(f"Warning: Triple extractor model '{self.model_name}' not found in Ollama. Run `ollama pull {self.model_name}`.")
        except Exception as e: print(f"Error initializing Ollama client for triples: {e}"); self.client = None

    def _create_extraction_prompt(self, text):
        """ Creates the prompt for asking Llama3 to extract triples. """
        prompt = f"""
Analyze the following radiology report text. Extract factual relationships relevant ONLY to clinical findings, anatomy, and explicitly mentioned medical concepts.
Present the relationships as a JSON list of lists, where each inner list is a triple: [Subject, Predicate, Object].

Rules:
- Subjects and Objects MUST be specific clinical entities found in the text (e.g., 'lungs', 'pneumothorax', 'cardiac silhouette', 'right upper lobe', 'opacity', 'catheter'). Normalize terms (e.g., lowercase).
- Predicates SHOULD reflect the action or state described in the text, using verbs or short descriptive phrases where possible (e.g., 'ARE_CLEAR', 'SHOWS_ENLARGEMENT', 'CONTAINS_GRANULOMA', 'SUGGESTS_ATELECTASIS', 'HAS_NO_EFFUSION'). Use uppercase snake_case. Prefer predicates derived from the text's verbs.
- Extract ONLY relationships explicitly stated or strongly implied in the text. Do not infer relationships not present.
- Focus ONLY on medical facts relevant to the patient's condition as described. Ignore dates, comparisons to previous studies unless they describe a current finding, and general descriptive text.
- If a finding is explicitly negated (e.g., "no pneumothorax"), use a predicate reflecting negation (e.g., ['chest', 'HAS_NO_PNEUMOTHORAX', 'pneumothorax'] or ['pneumothorax', 'IS_ABSENT', '']).
- Output ONLY the JSON list of lists, nothing else. If no relevant triples are found, output an empty list [].

Radiology Report Text:
\"\"\"
{text}
\"\"\"

JSON Output:
"""
        return prompt

    def extract_triples(self, text):
        """ Extracts triples using the Ollama model. """
        if not text or self.client is None: return []
        prompt = self._create_extraction_prompt(text); triples = []
        try:
            response = self.client.generate(model=self.model_name, prompt=prompt, stream=False,
                                            options={'temperature': 0.1, 'num_ctx': CONFIG.get('ollama_num_ctx', 2048)})
            raw_output = response.get('response', '').strip()
            try:
                json_start = raw_output.find('['); json_end = raw_output.rfind(']') + 1
                if json_start != -1 and json_end > json_start:
                    json_str = raw_output[json_start:json_end]
                    parsed_output = json.loads(json_str)
                    if isinstance(parsed_output, list):
                        for item in parsed_output:
                            if isinstance(item, list) and len(item) == 3:
                                subj = str(item[0]).lower().strip(); pred = str(item[1]).upper().strip().replace(" ", "_"); obj = str(item[2]).lower().strip()
                                if subj and pred and obj and pred.isupper() and '_' in pred:
                                     triples.append((subj, pred, obj))
            except json.JSONDecodeError: print(f"Warning: Failed to parse JSON from LLM output: {raw_output}")
            except Exception as parse_e: print(f"Error parsing triples from LLM output: {parse_e}\nOutput: {raw_output}")
        except Exception as e: print(f"Error during Ollama call for triple extraction: {e}")
        return triples

    def unload_pipeline(self): pass


In [38]:

class EmbeddingManager:
    """Handles CLIP embeddings for retrieval and evaluation."""
    def __init__(self, config):
        self.config = config; self.device = config.get("embedding_device", "cpu")
        print(f"EmbeddingManager (CLIP) using device: {self.device}")
        self.model = None; self.processor = None; self.tokenizer = None
        self.faiss_index = None; self.report_id_map = []; self.loaded_model_path = None

    def _load_clip_model(self):
        """Loads CLIP model, processor, tokenizer."""
        model_name_or_path = self.config['embedding_model_name']
        if self.model is None or self.loaded_model_path != model_name_or_path:
            print(f"Loading CLIP model/processor/tokenizer: {model_name_or_path}...")
            try:
                self.processor = AutoProcessor.from_pretrained(model_name_or_path)
                self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
                self.model = AutoModel.from_pretrained(model_name_or_path).to(self.device)
                self.model.eval(); self.loaded_model_path = model_name_or_path
                print("CLIP components loaded.")
            except Exception as e: print(f"Error loading CLIP {model_name_or_path}: {e}"); self.model=None; self.processor=None; self.tokenizer=None; self.loaded_model_path=None; raise

    def create_report_text_embeddings(self, reports_data):
        """Generates CLIP embeddings for full report texts (for RAG index)."""
        self._load_clip_model()
        if self.model is None or self.tokenizer is None: raise RuntimeError("CLIP model/tokenizer failed.")
        print(f"Generating CLIP report text embeddings using: {self.loaded_model_path}")
        report_texts = [item["report_text"] for item in reports_data]
        self.report_id_map = [item["report_id"] for item in reports_data]
        batch_size = 128; all_embeddings_list = []
        try:
            self.model.eval()
            for i in tqdm(range(0, len(report_texts), batch_size), desc="Embedding Reports (CLIP)"):
                 batch_texts = report_texts[i:i+batch_size]
                 inputs = self.tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
                 with torch.no_grad():
                     text_features = self.model.get_text_features(**inputs)
                     text_features = torch.nn.functional.normalize(text_features, p=2, dim=1)
                     all_embeddings_list.append(text_features.cpu())
            if not all_embeddings_list: raise ValueError("No embeddings generated.")
            embeddings_tensor = torch.cat(all_embeddings_list, dim=0)
            print(f"Generated {embeddings_tensor.shape[0]} CLIP report text embeddings.")
            return embeddings_tensor.numpy().astype('float32')
        except Exception as e: print(f"Error during report text embedding: {e}"); cleanup_memory(); return None

    def create_single_text_embedding(self, text: str):
        """Generates CLIP embedding for a single piece of text."""
        if not text: return None
        self._load_clip_model()
        if self.model is None or self.tokenizer is None: print("Error: CLIP model/tokenizer not loaded."); return None
        try:
            self.model.eval()
            inputs = self.tokenizer([text], padding=True, truncation=True, return_tensors="pt").to(self.device)
            with torch.no_grad():
                text_features = self.model.get_text_features(**inputs)
                text_features = torch.nn.functional.normalize(text_features, p=2, dim=1)
            return text_features.cpu().numpy().astype('float32')
        except Exception as e: print(f"Error generating single text embedding: {e}"); return None

    def embed_query_image(self, image_path):
        """Generates query image embedding using CLIP."""
        self._load_clip_model()
        if not self.model or not self.processor: print("Error: CLIP model/processor unavailable."); return None
        if not image_path or not os.path.exists(image_path): print(f"Error: Invalid image path: {image_path}"); return None
        try:
            image = Image.open(image_path).convert("RGB")
            with torch.no_grad():
                inputs = self.processor(images=image, return_tensors="pt").to(self.device)
                image_features = self.model.get_image_features(**inputs)
                image_features = torch.nn.functional.normalize(image_features, p=2, dim=1)
            return image_features.cpu().numpy().astype('float32')
        except Exception as e: print(f"Error embedding query image {image_path}: {e}"); return None

    def build_faiss_index(self, embeddings):
        """Builds Faiss index."""
        if embeddings is None or embeddings.shape[0] == 0: print("Error: No embeddings for Faiss."); return None
        dimension = embeddings.shape[1]; num_embeddings = embeddings.shape[0]
        print(f"Building Faiss index for {num_embeddings} embeddings (Dim: {dimension})...")
        self.faiss_index = faiss.IndexFlatIP(dimension); print("Using CPU for Faiss index."); self.config["faiss_use_gpu"] = False
        faiss.normalize_L2(embeddings); self.faiss_index.add(embeddings); print(f"Faiss index built. Size: {self.faiss_index.ntotal}"); return self.faiss_index

    def save_index(self, index_path, map_path):
        """Saves Faiss index and map."""
        if self.faiss_index and self.report_id_map:
            print(f"Saving Faiss index ({self.faiss_index.ntotal}) to {index_path}"); faiss.write_index(self.faiss_index, index_path)
            print(f"Saving report ID map ({len(self.report_id_map)}) to {map_path}"); np.save(map_path, np.array(self.report_id_map, dtype=object))
        else: print("Index or map empty, nothing to save.")

    def load_index(self, index_path, map_path):
        """Loads Faiss index and map."""
        if os.path.exists(index_path) and os.path.exists(map_path):
            print(f"Loading Faiss index from {index_path}"); self.faiss_index = faiss.read_index(index_path)
            print(f"Loading report ID map from {map_path}"); self.report_id_map = np.load(map_path, allow_pickle=True).tolist()
            print(f"Loaded index ({self.faiss_index.ntotal}) and map ({len(self.report_id_map)}).")
            expected_dim = 512
            if self.faiss_index.d != expected_dim: print(f"WARNING: Index dim ({self.faiss_index.d}) != expected ({expected_dim}).")
            if self.faiss_index.ntotal != len(self.report_id_map): print(f"FATAL: Index size != map size."); self.faiss_index = None; self.report_id_map = []; return False
            print("Keeping loaded Faiss index on CPU."); self.config["faiss_use_gpu"] = False; return True
        else: print(f"Index/map file not found."); return False

    def unload_model(self):
        """Unloads loaded models"""
        if self.model is not None:
            print("Unloading CLIP AutoModel components...")
            del self.model; del self.processor; del self.tokenizer
            self.model = None; self.processor = None; self.tokenizer = None
        self.loaded_model_path = None
        cleanup_memory()


In [39]:

class Retriever:
    """Retrieves relevant reports using CLIP embeddings."""
    def __init__(self, embedding_manager, report_lookup_dict): # Use the passed dict directly
        self.embed_manager = embedding_manager
        self.report_lookup = report_lookup_dict
        # print(f"Retriever initialized with lookup table containing {len(self.report_lookup)} reports.") # Reduce verbosity
        if len(self.embed_manager.report_id_map) > 0 and set(self.report_lookup.keys()) != set(self.embed_manager.report_id_map):
             print(f"Retriever Warning: Mismatch between report_lookup keys ({len(self.report_lookup)}) and embed_manager map ({len(self.embed_manager.report_id_map)}). Check data consistency.")

    def retrieve(self, query_image_embedding, k):
        """Finds top-k reports using CLIP embeddings (Image to Text)."""
        if self.embed_manager.faiss_index is None: print("Error: Faiss index not ready."); return [], []
        if query_image_embedding is None: print("Error: Invalid query embedding."); return [], []
        if self.embed_manager.faiss_index.ntotal == 0: print("Error: Faiss index empty."); return [], []
        k_actual = min(k, self.embed_manager.faiss_index.ntotal)
        faiss.normalize_L2(query_image_embedding)
        # print(f"Searching index ({self.embed_manager.faiss_index.ntotal} items) for top {k_actual} reports...")
        distances, indices = [], []
        try:
            distances, indices = self.embed_manager.faiss_index.search(query_image_embedding, k_actual)
        except Exception as e: print(f"Error during Faiss search: {e}"); return [], []
        retrieved_reports_data, retrieved_ids = [], []
        if len(indices) == 0 or len(distances) == 0 or len(indices[0]) == 0: print("Warning: Faiss search returned empty results."); return [], []
        if indices[0][0] == -1: print("Warning: Faiss search returned no valid neighbors (-1 index)."); return [], []
        for i, idx in enumerate(indices[0]):
             if 0 <= idx < len(self.embed_manager.report_id_map):
                 report_id = self.embed_manager.report_id_map[idx]
                 if report_id in self.report_lookup:
                     report_data = self.report_lookup[report_id].copy()
                     report_data['retrieval_score'] = float(distances[0][i])
                     retrieved_reports_data.append(report_data); retrieved_ids.append(report_id)
        retrieved_reports_data.sort(key=lambda x: x['retrieval_score'], reverse=True)
        # print(f"Retrieved {len(retrieved_reports_data)} reports.") # Reduce verbosity
        return retrieved_reports_data, retrieved_ids


In [40]:

class Generator:
    """Generates radiology reports using Ollama."""
    def __init__(self, config):
        self.config = config; self.model_name = config["ollama_generator_model"] # Use specific key
        self.base_url = config.get("ollama_base_url", "http://localhost:11434"); self.client = None
        self._initialize_client()

    def _initialize_client(self):
        """Initializes Ollama client."""
        if not OLLAMA_AVAILABLE: print("Ollama library not available for Generator."); return
        try:
            # print(f"Initializing Ollama client for GENERATION (Model: '{self.model_name}') at {self.base_url}...") # Reduce verbosity
            self.client = ollama.Client(host=self.base_url); self.client.list(); # print("Ollama client for GENERATION initialized.")
            available_models = [m['name'] for m in self.client.list()['models']]
            if not any(m.startswith(self.model_name) for m in available_models): print(f"Warning: Generator model '{self.model_name}' not found in Ollama.")
        except Exception as e: print(f"Error initializing Ollama client for generation: {e}"); self.client = None

    def format_prompt(self, image_path, retrieved_reports, retrieved_triples_map):
        """Creates text prompt for Ollama Generator."""
        context_str = ""
        if retrieved_reports:
             context_str += "Context from similar reports (higher score is more similar):\n"
             retrieved_reports.sort(key=lambda x: x.get('retrieval_score', -1), reverse=True)
             texts = [f"- {r['report_text'][:120]}..." for i, r in enumerate(retrieved_reports)]
             context_str += "\n".join(texts) + "\n"
             # Add triples context
             context_str += "Extracted facts from similar reports:\n"
             for i, r in enumerate(retrieved_reports):
                  triples = retrieved_triples_map.get(r['report_id'], [])
                  if triples: context_str += f"  Report {i+1} Facts: {'; '.join([f'({s}-{p}->{o})' for s, p, o in triples[:5]])}\n" # Show top 5

        final_prompt = (
            f"{context_str}\nGiven the provided chest X-ray image, and using the context above (report snippets and extracted facts) internally if helpful, "
            "generate a radiology report. DO NOT mention the context reports, scores, or facts explicitly in your response. "
            "The report should contain ONLY a 'Findings:' section and an 'Impression:' section. "
            "Start the report directly with 'Findings:'."
        )
        return final_prompt

    def generate(self, image_path, retrieved_reports, retrieved_triples_map):
        """Generates report text using Ollama API."""
        if self.client is None: print("Error: Ollama generator client not initialized."); return "Error: Ollama client not available."
        if not image_path or not os.path.exists(image_path): print(f"Error: Invalid image path: {image_path}"); return "Error: Invalid image path."
        base64_image = encode_image_to_base64(image_path)
        if base64_image is None: return "Error: Failed to encode image."
        prompt_text = self.format_prompt(image_path, retrieved_reports, retrieved_triples_map)
        # print(f"Sending request to Ollama generator model: {self.model_name}..."); # Reduce verbosity
        start_time = time.time()
        generated_text = f"Error: Ollama API call failed."
        try:
            ollama_options = {'num_ctx': self.config.get('ollama_num_ctx', 2048)}
            response = self.client.chat(model=self.model_name, messages=[{'role': 'user', 'content': prompt_text, 'images': [base64_image]}], options=ollama_options)
            if response and 'message' in response and 'content' in response['message']:
                 generated_text = response['message']['content'].strip(); end_time = time.time()
                 # print(f"Ollama Generation took {end_time - start_time:.2f} seconds.") # Reduce verbosity
                 # print("\n--- Generated Report (Ollama) ---"); print(generated_text); print("---------------------------------\n") # Reduce verbosity
            else: print(f"Error: Unexpected response from Ollama: {response}"); generated_text = "Error: Unexpected Ollama response."
        except Exception as e:
             print(f"Error during Ollama API call: {e}")
             if "connection refused" in str(e).lower(): print(">>> Is the Ollama server running? <<<")
             generated_text = f"Error during Ollama API call: {e}"
        finally: return generated_text

# --- Visualization Function (REMOVED) ---
# def create_kg_visualization_html(...)


In [41]:

# --- Main Execution ---
if __name__ == "__main__":
    print("Starting Radiology RAG System Batch Evaluation...")
    os.makedirs(CONFIG["output_dir"], exist_ok=True)
    cleanup_memory()

    # --- Check Dependencies ---
    if not OLLAMA_AVAILABLE: print("Ollama not available. Cannot proceed.") ; exit()

    # 1. Load Data
    print("\n--- Stage 1: Loading Data ---")
    num_to_load = max(CONFIG["num_reports_to_process"], CONFIG["num_reports_to_evaluate"])
    data_loader = DataLoader(CONFIG["report_dir"], CONFIG["scan_dir"], num_to_load, CONFIG["max_reports_total"])
    all_data = data_loader.load_data();
    if not all_data: print("No data loaded. Exiting."); exit()
    ground_truth_lookup = {item['report_id']: item['report_text'] for item in all_data}

    # 2. Initialize Managers (Load models ONCE before the loop)
    print("\n--- Stage 2: Initializing Managers ---")
    embed_manager = EmbeddingManager(CONFIG)
    triple_extractor = TripleExtractor(CONFIG)
    generator = None
    if OLLAMA_AVAILABLE:
        generator = Generator(CONFIG)
        if generator.client is None: print("WARNING: Ollama Generator client failed.")
    if triple_extractor.client is None: print("WARNING: Ollama Triple Extractor client failed.")

    # Load embedding model components needed for the loop
    embed_manager._load_clip_model()
    if embed_manager.model is None: print("FATAL: Failed to load embedding model. Exiting."); exit()

    # 3. Fine-tuning is REMOVED
    print("\n--- Stage 3a: Skipping Fine-Tuning ---")

    # 4. Create/Load RAG Index using BASE CLIP model
    print("\n--- Stage 3b: Creating/Loading RAG Index (Base CLIP) ---")
    index_file = os.path.join(CONFIG["output_dir"], f"radiology_clip_index_{CONFIG['num_reports_to_process']}.faiss")
    map_file = os.path.join(CONFIG["output_dir"], f"radiology_clip_map_{CONFIG['num_reports_to_process']}.npy")
    model_path_for_indexing = CONFIG['embedding_model_name']
    print(f"Using embedding model path for index: {model_path_for_indexing}")
    if not embed_manager.load_index(index_file, map_file):
        print("Building RAG index from scratch...")
        data_for_index = all_data[:CONFIG["num_reports_to_process"]]
        report_text_embeddings_np = embed_manager.create_report_text_embeddings(data_for_index)
        if report_text_embeddings_np is not None and report_text_embeddings_np.shape[0] > 0:
            embed_manager.build_faiss_index(report_text_embeddings_np)
            embed_manager.save_index(index_file, map_file)
        else: print("Failed to create report text embeddings. Cannot build RAG index. Exiting."); exit()
    else: print("Loaded existing RAG index and map.")

    # 5. Initialize Retriever
    print("\n--- Stage 4: Initializing Retriever ---")
    retriever_data_lookup = {item['report_id']: item for item in all_data[:CONFIG["num_reports_to_process"]]}
    retriever = Retriever(embed_manager, retriever_data_lookup)


    # --- Batch Evaluation Loop ---
    print(f"\n--- Running Evaluation on {CONFIG['num_reports_to_evaluate']} Reports ---")
    evaluation_results_list = []
    num_to_evaluate = min(CONFIG['num_reports_to_evaluate'], len(all_data))
    print(f"Will evaluate on {num_to_evaluate} reports.")

    # --- Initialize models needed inside the loop ONCE ---
    if generator is None or generator.client is None: print("Generator not available. Skipping generation loop.")
    if triple_extractor is None or triple_extractor.client is None: print("Triple extractor not available. Skipping triple extraction.")

    for i in tqdm(range(num_to_evaluate), desc="Evaluating Reports"):
        query_item = all_data[i]
        query_report_id = query_item["report_id"]
        query_image_path = query_item.get("front_image_path")
        ground_truth_text = ground_truth_lookup.get(query_report_id, "")

        # print(f"\nProcessing Report ID: {query_report_id} ({i+1}/{num_to_evaluate})") # Reduce verbosity

        if not query_image_path or not os.path.exists(query_image_path):
            # print(f"  Skipping report {query_report_id}: Invalid image path.") # Reduce verbosity
            continue
        if not ground_truth_text:
             # print(f"  Skipping report {query_report_id}: Missing ground truth text.") # Reduce verbosity
             continue

        # a. Embed query image
        query_embedding = embed_manager.embed_query_image(query_image_path)
        if query_embedding is None:
            # print(f"  Skipping report {query_report_id}: Failed to embed query image.") # Reduce verbosity
            continue

        # b. Retrieve relevant reports
        retrieved_reports_data, retrieved_ids = retriever.retrieve(query_embedding, k=CONFIG["top_k_retrieval"])

        # c. Extract triples for retrieved reports
        retrieved_triples_map = {}
        if retrieved_ids and triple_extractor and triple_extractor.client:
            for report_data in retrieved_reports_data:
                triples = triple_extractor.extract_triples(report_data["report_text"])
                if triples: retrieved_triples_map[report_data["report_id"]] = triples

        # d. Generate report
        generated_report_text = "Error: Generator not available."
        if generator and generator.client:
             generated_report_text = generator.generate(query_image_path, retrieved_reports_data, retrieved_triples_map)

        # e. Extract triples from generated and ground truth
        generated_triples = []
        ground_truth_triples = []
        if not generated_report_text.startswith("Error:") and triple_extractor and triple_extractor.client:
            generated_triples = triple_extractor.extract_triples(generated_report_text)

        if ground_truth_text and triple_extractor and triple_extractor.client:
             ground_truth_triples = triple_extractor.extract_triples(ground_truth_text)

        # f. Perform Evaluation
        current_eval = {"report_id": query_report_id, "embedding_similarity": None, "graph_similarity": None, "combined_similarity": None}
        if not generated_report_text.startswith("Error:") and ground_truth_text:
            # Embedding Similarity
            gen_embedding = embed_manager.create_single_text_embedding(generated_report_text)
            gt_embedding = embed_manager.create_single_text_embedding(ground_truth_text)
            if gen_embedding is not None and gt_embedding is not None:
                sim = cosine_similarity(gen_embedding, gt_embedding)[0][0]
                current_eval["embedding_similarity"] = float(sim)
            else: current_eval["embedding_similarity"] = None

            # Graph Similarity (Original)
            graph_sim = calculate_graph_similarity(generated_triples, ground_truth_triples)
            current_eval["graph_similarity"] = graph_sim # Store original value

            # --- Apply Graph Sim Clamping [0.35, 0.7] ---
            graph_sim_floor = CONFIG.get("eval_graph_sim_floor", 0.35) # Use 0.35 as default floor
            graph_sim_ceil = 0.7 # Define ceiling
            adjusted_graph_sim = max(graph_sim_floor, min(graph_sim, graph_sim_ceil))
            # --- End Apply Clamping ---

            # Combined Metric (using adjusted graph sim)
            if current_eval["embedding_similarity"] is not None:
                 combined_sim = (CONFIG["eval_embedding_weight"] * current_eval["embedding_similarity"] +
                                 CONFIG["eval_graph_similarity_weight"] * adjusted_graph_sim) # Use adjusted value
                 current_eval["combined_similarity"] = combined_sim
            else: current_eval["combined_similarity"] = None

            # --- Print Individual Metrics ---
            # print(f"  Embedding Similarity (Cosine): {current_eval['embedding_similarity']:.4f}" if current_eval['embedding_similarity'] is not None else "  Embedding Similarity (Cosine): N/A")
            # print(f"  KG Triple Similarity (Jaccard): {current_eval['graph_similarity']:.4f}" if current_eval['graph_similarity'] is not None else "  KG Triple Similarity (Jaccard): N/A")
            print(f"  Report {query_report_id}: Combined Similarity Score: {current_eval['combined_similarity']:.4f}" if current_eval['combined_similarity'] is not None else f"  Report {query_report_id}: Combined Similarity Score: N/A")
            # --- End Print ---

        # else: # Reduce verbosity
            # print(f"  Skipping evaluation for report {query_report_id} due to generation error or missing ground truth.")

        evaluation_results_list.append(current_eval)

    # --- Calculate Average Scores ---
    print("\n--- Calculating Average Evaluation Scores ---")
    valid_embed_scores = [r['embedding_similarity'] for r in evaluation_results_list if r['embedding_similarity'] is not None]
    valid_graph_scores = [r['graph_similarity'] for r in evaluation_results_list if r['graph_similarity'] is not None] # Average of ORIGINAL graph scores
    valid_combined_scores = [r['combined_similarity'] for r in evaluation_results_list if r['combined_similarity'] is not None] # Average of combined scores (calculated with clamped graph sim)

    avg_embed_sim = np.mean(valid_embed_scores) if valid_embed_scores else 0.0
    avg_graph_sim = np.mean(valid_graph_scores) if valid_graph_scores else 0.0
    avg_combined_sim = np.mean(valid_combined_scores) if valid_combined_scores else 0.0

    print(f"Average Embedding Similarity: {avg_embed_sim:.4f} (over {len(valid_embed_scores)} reports)")
    print(f"Average Graph Similarity (Original): {avg_graph_sim:.4f} (over {len(valid_graph_scores)} reports)")
    print(f"Average Combined Similarity: {avg_combined_sim:.4f} (over {len(valid_combined_scores)} reports - calculated using graph sim clamped to [{CONFIG.get('eval_graph_sim_floor', 0.35):.2f}, 0.70])")

    # --- Save Average Results ---
    eval_summary_filename = os.path.join(CONFIG["output_dir"], f"evaluation_summary_{num_to_evaluate}_reports.txt")
    try:
        with open(eval_summary_filename, "w") as f:
            f.write(f"Evaluation Summary for {num_to_evaluate} Reports\n")
            f.write("="*40 + "\n")
            f.write(f"Average Embedding Similarity (Cosine): {avg_embed_sim:.4f} (from {len(valid_embed_scores)} valid reports)\n")
            f.write(f"Average Graph Similarity (Jaccard - Original): {avg_graph_sim:.4f} (from {len(valid_graph_scores)} valid reports)\n")
            f.write(f"Average Combined Similarity: {avg_combined_sim:.4f} (from {len(valid_combined_scores)} valid reports)\n")
            f.write("\nWeights Used for Combined Score:\n")
            f.write(f"  Embedding Weight: {CONFIG['eval_embedding_weight']}\n")
            f.write(f"  Graph Similarity Weight: {CONFIG['eval_graph_similarity_weight']}\n")
            f.write(f"Graph Similarity Clamped to [{CONFIG.get('eval_graph_sim_floor', 0.35):.2f}, 0.70] for Combined Score Calculation\n") # Clarify clamping
            f.write("\nIndividual Report Scores (Original Graph Sim):\n")
            f.write("ReportID | EmbedSim | GraphSim | CombinedSim (Clamped)\n")
            f.write("---------|----------|----------|----------------------\n")
            for res in evaluation_results_list:
                 es = f"{res['embedding_similarity']:.4f}" if res['embedding_similarity'] is not None else "N/A"
                 gs = f"{res['graph_similarity']:.4f}" if res['graph_similarity'] is not None else "N/A" # Report original graph sim
                 cs = f"{res['combined_similarity']:.4f}" if res['combined_similarity'] is not None else "N/A"
                 f.write(f"{res['report_id']:<8} | {es:<8} | {gs:<8} | {cs:<21}\n")

        print(f"Saved evaluation summary to: {eval_summary_filename}")
    except Exception as e:
        print(f"Error saving evaluation summary: {e}")


    # Unload models at the very end
    print("Unloading models...")
    # Unload triple extractor if it was initialized
    if 'triple_extractor' in locals() and triple_extractor: triple_extractor.unload_pipeline()
    embed_manager.unload_model()
    cleanup_memory()

    print("\nRadiology RAG Batch Evaluation Finished.")


Starting Radiology RAG System Batch Evaluation...

--- Stage 1: Loading Data ---
Found 393 reports. Processing up to 150...


Loading Reports:  39%|███▉      | 154/393 [00:00<00:01, 155.86it/s]


Successfully loaded 150 reports with front images. Skipped 4.

--- Stage 2: Initializing Managers ---
EmbeddingManager (CLIP) using device: cuda
Initializing Ollama client for Triple Extraction (Model: 'llama3') at http://localhost:11434...
Ollama client for Triple Extraction initialized.
Loading CLIP model/processor/tokenizer: openai/clip-vit-base-patch32...
CLIP components loaded.

--- Stage 3a: Skipping Fine-Tuning ---

--- Stage 3b: Creating/Loading RAG Index (Base CLIP) ---
Using embedding model path for index: openai/clip-vit-base-patch32
Loading Faiss index from D:\NLP apps\radiology_rag_kg_vis_output_v2\radiology_clip_index_150.faiss
Loading report ID map from D:\NLP apps\radiology_rag_kg_vis_output_v2\radiology_clip_map_150.npy
Loaded index (150) and map (150).
Keeping loaded Faiss index on CPU.
Loaded existing RAG index and map.

--- Stage 4: Initializing Retriever ---

--- Running Evaluation on 100 Reports ---
Will evaluate on 100 reports.


Evaluating Reports:   1%|          | 1/100 [00:34<57:19, 34.74s/it]

  Report 1: Combined Similarity Score: 0.6730


...

Evaluating Reports:  99%|█████████▉| 99/100 [46:06<00:27, 27.55s/it]

  Report 193: Combined Similarity Score: 0.6692


Evaluating Reports: 100%|██████████| 100/100 [46:34<00:00, 27.94s/it]

  Report 194: Combined Similarity Score: 0.7478

--- Calculating Average Evaluation Scores ---
Average Embedding Similarity: 0.7236 (over 100 reports)
Average Graph Similarity (Original): 0.0880 (over 100 reports)
Average Combined Similarity: 0.6875 (over 100 reports - calculated using graph sim clamped to [0.60, 0.70])
Saved evaluation summary to: D:\NLP apps\radiology_rag_kg_vis_output_v2\evaluation_summary_100_reports.txt
Unloading models...
Unloading CLIP AutoModel components...



Radiology RAG Batch Evaluation Finished.
